In [ ]:
# Import modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import plotly.graph_objs as go 
from plotly.offline import init_notebook_mode,iplot


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
init_notebook_mode(connected=True)

In [ ]:
# Load data
df = pd.read_csv('data/house_data.csv')

In [ ]:
# reading the data
df.info()
df.shape
df.tail()

In [ ]:
# drop NaN, Price, URL, and assign to df1
def metric_deletion(x):
    x.dropna(axis='rows',inplace=True)
    x = x[x.Bedrooms != '-- bds']
    x = x[x.Bathrooms != '-- ba']
    x = x[x.Size != '-- sqft']
    x.drop(['URL', 'Price'], axis=1,inplace=True)
    return x
df1 = metric_deletion(df)
df1.head()

In [ ]:
# Converting Bathrooms into float
df1.Bathrooms = df1.Bathrooms.str.replace(' ba','').astype('float')
# Converting Bedrooms into float
df1.Bedrooms = df1.Bedrooms.str.replace(' bds','').astype('float')

In [ ]:
# Converting Size to float
def filt_size(s):
    s= s.replace(',','')
    s =s.replace(' sqft','')
    return float(s)
df1.Size = df1.Size.apply(filt_size)

In [ ]:
# Spliting Address into Street, City, State, ZipCode, and drop the Address
df1.Address = df1.Address.astype('string')
df1['Street']= df1.Address.apply(lambda x: x.split(', ')[0])
df1['City']= df1.Address.apply(lambda x: x.split(', ')[1])
df1['State']= df1.Address.apply(lambda x: (x.split(', ')[-1]).split(' ')[0])
df1['ZipCode']= df1.Address.apply(lambda x: (x.split(', ')[-1]).split(' ')[1])
df2 = df1.drop(['Address'],axis=1)


In [ ]:
#reset the index
df2.reset_index(inplace=True,drop=True)

In [ ]:
# Found two rows of abnormal values, so found exact address on google and replace with the right values
df2.loc[28709:28711, 'State']= 'AZ'
df2.loc[28709:28711, 'ZipCode']= '85260'

In [ ]:
# Converting the columns as strings for further cleaning
df2[['Street','City','State','ZipCode']].astype('string')

In [ ]:
# Finding the weird ZipCode, it is in Canada
df2.loc[df2.ZipCode == 'N9V']

In [ ]:
# Dropping the row
df2.drop(df2.iloc[38379].name,inplace=True)

In [ ]:
# Now the ZipCode can be converted to Integer
df2.ZipCode = df2.ZipCode.astype('int')


In [ ]:
#convert Sale Status into house Types
house_status = list(df2['Sale Status'].unique())
house_type = ['House','Townhouse','Multifamily', 'Condo', 'Others', 'Apartment']
df2['Sale Status'] = df2['Sale Status'].map(dict(zip(house_status,house_type)))
df2.rename(columns={"Sale Status": "Type"}, inplace=True)


In [ ]:
# Street Column might not be useful
df3 = df2.drop(['Street'], axis=1)

In [ ]:
# Change Raw Price column name to Price, create perSqft column
df3.rename(columns={"Raw Price":"Price"}, inplace=True)
df3['perSqFt'] = df3.Price / df3.Size

In [ ]:
# Due to previously dropping rows, reset index again
df3 = df3.reset_index(drop=True)


In [ ]:
# Assign Regions
west = ['CA', 'NV', 'AK', 'WA' , 'OR', 'ID', 'MT', 'WY', 'UT', 'CO', 'AZ', 'NM', 'HI']
midwest = ['ND', 'WI','SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'OH','MI']
north = ['PA', 'NY', 'NH', 'MA', 'CT', 'ME', 'DC', 'NJ', 'RI']
south = ['TX', 'OK', 'AR', 'LA', 'MS', 'AL', 'TN', 'KY', 'WV', 'VA', 'MD', 'DE', 'NC', 'SC', 'GA', 'FL']
full_state_list = west + midwest + north + south

# Creating function to assign regions
def find_region(state):
    if state in west:
        state = 'West'
    elif state in north:
        state = 'North'
    elif state in south:
        state = 'South'
    elif state in midwest:
        state = 'MidWest'
    return state

# Create Region column
df3['Region'] = df3.State.apply(find_region)

In [408]:
pd.set_option('display.float_format', '{:.2f}'.format)
pd.pivot_table(df3, values=['perSqFt'], index=['State'], columns=['Type'])


perSqFt                                             
Type  Apartment   Condo  House Multifamily  Others Townhouse
State                                                       
AK          NaN  175.66 217.43      130.48  114.25    175.70
AL          NaN  223.22 124.42      197.43     NaN    168.00
AR          NaN  160.26 142.64      104.64   37.77    140.67
AZ       333.11  158.56 275.54    12829.84 1283.75    260.90
CA          NaN  612.58 514.14      493.40  149.31    510.89
CO          NaN  449.90 318.69      343.37   76.11    335.75
CT          NaN  242.76 256.85      109.60     NaN       NaN
DC          NaN  632.31 565.86      548.56     NaN    493.12
DE       235.90  231.23 189.45      113.23     NaN    104.08
FL          NaN  251.37 262.30      140.71   52.24    217.55
GA          NaN  236.55 146.01      122.88   49.30    214.57
HI          NaN 1283.64 681.20         NaN     NaN       NaN
IA          NaN  145.63 101.97       42.35     NaN     38.90
ID          NaN  299.77 277.96      249.76  119.84    258.89
IL          NaN  210.57 163.51       86.52  108.42    183.46
IN          NaN  180.55 106.16       48.00   66.04       NaN
KS          NaN   87.65 124.75       83.06     NaN    131.79
KY          NaN  220.35 196.08      116.37     NaN    193.48
LA          NaN  273.38 132.46      170.70   60.57    246.03
MA          NaN  767.97 477.66      326.84  136.55       NaN
MD       143.41  240.58 166.03      141.18     NaN    160.04
ME          NaN  628.44 343.93      462.25     NaN    408.65
MI          NaN  149.26  95.71      112.26     NaN    134.78
MN          NaN  281.37 224.36      170.73     NaN    186.65
MO          NaN  180.21 129.51       73.22     NaN    111.35
MS          NaN  169.27  99.85       48.55     NaN     88.37
MT          NaN  220.03 206.32         NaN   80.63    157.81
NC          NaN  346.57 234.34       74.42  106.40    219.51
ND          NaN  138.07 121.00      102.58     NaN    116.88
NE          NaN  272.11 140.29      107.31   36.19    164.96
NH          NaN  251.02 234.07         NaN     NaN       NaN
NJ        91.25  529.59 204.87     3712.10     NaN    112.19
NM          NaN  182.74 165.60      132.71   90.39    177.23
NV          NaN  184.83 252.85         NaN  124.33    196.25
NY      1948.00  922.74 327.82      248.27   68.47   1756.35
OH          NaN  200.33  95.02       57.56     NaN    217.41
OK          NaN  132.58 148.36      119.61   98.21    266.56
OR          NaN  253.06 309.65      184.84  159.09    293.80
PA       238.26  306.24 174.21      127.97  118.39    217.02
RI          NaN  239.65 205.27      130.38   90.64    200.12
SC          NaN  206.52 223.68      128.16   82.90    187.22
SD          NaN  191.93 183.06         NaN   98.80    216.34
TN          NaN  288.01 245.27      188.42  166.31    235.13
TX       157.71  157.39 147.95      157.37   97.69    122.54
UT          NaN  265.59 220.50      213.02   59.37    198.47
VA       165.70  239.13 413.64      177.00  150.90    195.00
WA          NaN  450.28 270.26      207.50  116.32    308.61
WI          NaN  215.68 173.93      142.85     NaN       NaN

In [ ]:
'''
NOT WORKING CORRECTLY


data = dict(type = 'choropleth',colorscale='Portland',locations=df3['State'],locationmode='USA-states', z=df3['Price'],text=df3['Price'], colorbar={'title':'perSqFt'})
choromap = go.Figure(data = [data],layout = dict(geo = {'scope':'usa'}))

iplot(choromap,validate=False)
'''

In [ ]:
# for GeoPandas later use
# df2 = df2[df2['ZipCode'].between(10000,99999,inclusive='both')]